In [1]:
import numpy as np

#Class extracted from finnthesizer.py
class BNNWeightReader:
  def __init__(self, paramFile, interleaveChannels):
    self.paramDict = np.load(paramFile)
    self.currentParamInd = 0
    self.interleaveChannels = interleaveChannels
    self.numInterleaveChannels = 0
    
  def __getCurrent(self):
    print("arr_{} ".format(self.currentParamInd),end = '')
    ret =  self.paramDict["arr_" + str(self.currentParamInd)]
    self.currentParamInd += 1
    return ret
    
  def readWeightsRaw(self):
    w = self.__getCurrent()
    return w
    
  def readBatchNormLayerRaw(self):
    bias = self.__getCurrent()
    beta = self.__getCurrent()
    gamma = self.__getCurrent()
    mean = self.__getCurrent()
    invstd = self.__getCurrent()
    return (bias, beta, gamma, mean, invstd)
    
  # read a fully connected layer plus batchnorm, binarize and convert to
  # positive threshold form, returning (bin weight matrix, thresholds)
  # the returned bin weight matrix has neurons along rows and is suitable
  # to be packed into BNN mems using BNNProcElemMem
  def readFCBNComplex(self, WPrecisions_fract, APrecisions_fract, IPrecisions_fract, WPrecisions_int, APrecisions_int, IPrecisions_int, numThresBits=16, numThresIntBits=None):
    WPrecision = WPrecisions_fract + WPrecisions_int
    APrecision = APrecisions_fract + APrecisions_int
    IPrecision = IPrecisions_fract + IPrecisions_int
    weights = self.readWeightsRaw()
    (bias, beta, gamma, mean, invstd) = self.readBatchNormLayerRaw()

    if WPrecision==1 and APrecision==1 and IPrecision==1:
        (Wb, T) = makeFCBNComplex(weights, bias, beta, gamma, mean, invstd, WPrecisions_int, WPrecisions_fract, use_rowmajor=True)
    elif (APrecision==1):
        (Wb, T) = makeFCBNComplex(weights, bias, beta, gamma, mean, invstd, WPrecisions_int, WPrecisions_fract, use_rowmajor=True, usePopCount=False)
    else:
        (Wb, T) = makeFCBNComplex_QNN(weights, bias, beta, gamma, mean, invstd, WPrecisions_fract, APrecisions_fract, WPrecisions_int, APrecisions_int, True, numThresBits, numThresIntBits)
    # if the interleave flag is set, permute elements in each row
    if self.interleaveChannels and self.numInterleaveChannels != 0:
      print("Interleaving %d channels in fully connected layer..." % self.numInterleaveChannels)
      pixPerChan = Wb.shape[1] / self.numInterleaveChannels
      if (APrecisions_fract == 0):
        Wb_perm = np.zeros(Wb.shape, dtype=np.int)
      else:
        Wb_perm = np.zeros(Wb.shape, dtype=np.float)
      for r in range(Wb.shape[0]):
        for chan in range(self.numInterleaveChannels):
          for cpix in range(pixPerChan):
            Wb_perm[r][cpix*self.numInterleaveChannels + chan] = Wb[r][chan*pixPerChan + cpix]
      Wb = Wb_perm
      # set interleave to zero once we go past this fc layer
      self.numInterleaveChannels = 0

    return (Wb, T)

    # read a fully connected layer without batchnorm and without using thresholds, 
    # returning bin weight matrix
    # the returned bin weight matrix has neurons along rows and is suitable
    # to be packed into BNN mems using BNNProcElemMem    
  def readFCBNComplex_no_thresholds(self, WPrecisions_fract, APrecisions_fract, IPrecisions_fract, WPrecisions_int, APrecisions_int, IPrecisions_int, numThresBits=16, numThresIntBits=None):
    WPrecision = WPrecisions_fract + WPrecisions_int
    APrecision = APrecisions_fract + APrecisions_int
    IPrecision = IPrecisions_fract + IPrecisions_int
    
    weights = self.readWeightsRaw()
    
    #fake the batchnorm params to use same make functions below
    bias   = np.zeros(weights.shape[1])    
    beta   = np.zeros(weights.shape[1])
    gamma  = np.ones(weights.shape[1])
    mean   = np.ones(weights.shape[1])
    invstd = np.ones(weights.shape[1])

    if (WPrecision == 1) and (APrecision == 1) and (IPrecision == 1):
        (Wb, T) = makeFCBNComplex(weights, bias, beta, gamma, mean, invstd, WPrecisions_int, WPrecisions_fract, use_rowmajor=True)
    elif (APrecision==1):
        (Wb, T) = makeFCBNComplex(weights, bias, beta, gamma, mean, invstd, WPrecisions_int, WPrecisions_fract, use_rowmajor=True, usePopCount=False)
    else:
        (Wb, T) = makeFCBNComplex_QNN(weights, bias, beta, gamma, mean, invstd, WPrecisions_fract, APrecisions_fract, WPrecisions_int, APrecisions_int, True, numThresBits, numThresIntBits)
    
    # if the interleave flag is set, permute elements in each row
    if self.interleaveChannels and self.numInterleaveChannels != 0:
        print ("Interleaving %d channels in fully connected layer..." % self.numInterleaveChannels)
        pixPerChan = Wb.shape[1] / self.numInterleaveChannels
        if (APrecisions_fract == 0):
            Wb_perm = np.zeros(Wb.shape, dtype=np.int)
        else:
            Wb_perm = np.zeros(Wb.shape, dtype=np.float)
        for r in range(Wb.shape[0]):
            for chan in range(self.numInterleaveChannels):
                for cpix in range(pixPerChan):
                    Wb_perm[r][cpix*self.numInterleaveChannels + chan] = Wb[r][chan*pixPerChan + cpix]
        Wb = Wb_perm
        # set interleave to zero once we go past this fc layer
        self.numInterleaveChannels = 0
    
    return (Wb, T)
    
  # read a convolutional layer plus batchnorm, binarize and convert to
  # positive threshold form, returning (bin weight matrix, thresholds)
  # the returned bin weight matrix  is suitable to be packed into BNN mems 
  def readConvBNComplex(self, WPrecisions_fract, APrecisions_fract, IPrecisions_fract, WPrecisions_int, APrecisions_int, IPrecisions_int, usePopCount=True,numThresBits=16, numThresIntBits=None):
    weights = self.readWeightsRaw()
    (bias, beta, gamma, mean, invstd) = self.readBatchNormLayerRaw()
    # keep track of output channels for use in FC layer interleave
    self.numInterleaveChannels = weights.shape[0]
    (Wb, T) = makeConvBNComplex(weights, bias, beta, gamma, mean, invstd, self.interleaveChannels, WPrecisions_fract, APrecisions_fract, IPrecisions_fract, WPrecisions_int, APrecisions_int, IPrecisions_int, usePopCount=usePopCount, numThresBits=numThresBits, numThresIntBits=numThresIntBits)
    return (Wb, T)

In [3]:
reader_1w1a = BNNWeightReader("sfc_mnist-1w-1a.npz",False)

In [4]:
#We know beforehand the NN has 4 layers. Read description of 1w1a
for i in range(4):
    print("LAYER {}".format(i))
    weights1 = reader_1w1a.readWeightsRaw()
    (bias1, beta1, gamma1, mean1, invstd1) = reader_1w1a.readBatchNormLayerRaw()
    print("\nweights: {}\nbias: {}\nbeta: {}\ngamma: {}\nmean: {}\ninvstd: {}\n".format(weights1.shape,bias1.shape,
                                                                         beta1.shape,
                                                                         gamma1.shape,
                                                                         mean1.shape,
                                                                         invstd1.shape))


LAYER 0
arr_0 arr_1 arr_2 arr_3 arr_4 arr_5 
weights: (784, 256)
bias: (256,)
beta: (256,)
gamma: (256,)
mean: (256,)
invstd: (256,)

LAYER 1
arr_6 arr_7 arr_8 arr_9 arr_10 arr_11 
weights: (256, 256)
bias: (256,)
beta: (256,)
gamma: (256,)
mean: (256,)
invstd: (256,)

LAYER 2
arr_12 arr_13 arr_14 arr_15 arr_16 arr_17 
weights: (256, 256)
bias: (256,)
beta: (256,)
gamma: (256,)
mean: (256,)
invstd: (256,)

LAYER 3
arr_18 arr_19 arr_20 arr_21 arr_22 arr_23 
weights: (256, 10)
bias: (10,)
beta: (10,)
gamma: (10,)
mean: (10,)
invstd: (10,)

